Last Update: 3/14/2023

This notebook contains codes and results of VCL on permute MNIST dataset with Coresets using embeddings by VAE.



In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import torch
from torch import nn
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.distributions import kl_divergence, Normal
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
import pickle

from alg.coreset import Coreset, CoresetConfig

In [2]:
class BayesLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight_loc = nn.Parameter(torch.zeros(in_features, out_features))
        self.log_weight_scale = nn.Parameter(torch.zeros(in_features, out_features))
        
        self.bias_loc = nn.Parameter(torch.zeros(out_features))
        self.log_bias_scale = nn.Parameter(torch.zeros(out_features))
        
    def get_params(self):
        """
        return two tensors, obtaining by concatenating locs and scales together
        these parameters can be further used to calculate e.g, KL divergence (vectorizedly)
        """
        return (
                torch.cat([self.weight_loc.flatten(), self.bias_loc.flatten()]), 
                torch.cat([self.log_weight_scale.flatten(), self.log_bias_scale.flatten()])
               )
    
    def forward(self, x, x_is_sample, activition_fn, n_particles):
        """
        forward with local reparameterization tricks
        """
        ys = []
        if x_is_sample:
            assert n_particles == len(x)
            for _x in x:
                gamma = _x @ self.weight_loc + self.bias_loc
                delta2 = (_x ** 2) @ (torch.exp(self.log_weight_scale) ** 2) + torch.exp(self.log_bias_scale) ** 2
                y = gamma + torch.randn_like(gamma) * torch.sqrt(delta2 + 1e-6) 
                ys.append(activition_fn(y))
        else:
            for _ in range(n_particles):
                gamma = x @ self.weight_loc + self.bias_loc
                delta2 = (x ** 2) @ (torch.exp(self.log_weight_scale) ** 2) + torch.exp(self.log_bias_scale) ** 2
                y = gamma + torch.randn_like(gamma) * torch.sqrt(delta2 + 1e-6) 
                ys.append(activition_fn(y))
        return ys
    
    def forward_MLE(self, x, activition_fn):
        y = x @ self.weight_loc + self.bias_loc
        return activition_fn(y)


In [3]:
# Implement Gonzalez Algorithm for k-Center Clustering for Coreset selection

class KCenter:
    def __init__(self, K):
        self.K = K
    def fit_transform(self, x):
        # fit and return centers
        idx1 = np.random.choice(x.shape[0], 1)
        center = x[idx1] # (1, d)
        idx = [idx1.item()]
        min_distances_to_centers = np.full([1, x.shape[0]], np.inf)
        for k in tqdm(range(1, self.K)):
            distances = ((x[None, :, :] - center[:, None, :]) ** 2).sum(-1) # 1, N
            min_distances_to_centers = np.vstack([min_distances_to_centers, distances]).min(0)
            new_center_idx = np.argmax(min_distances_to_centers).item()
            idx.append(new_center_idx)
            center = x[[new_center_idx]]
        assert len(idx) == self.K
        return idx

In [4]:
class VCL_permute_MNIST(nn.Module):
    def __init__(self, previous_model, random_initialize):
        super().__init__()
        
        # "fully connected single-head networks with two hidden layers, where each layer contains 100 hidden units with ReLU activation"
        self.linear1 = BayesLinear(28*28, 100)
        self.linear2 = BayesLinear(100, 100)
        self.head = BayesLinear(100, 10)
        
        # intialize distributions as prior distributions
        # I believe there exist better ways to do this, but I am too lazy to think..(sorry)
        
        # define a layer dict
        self.layer_dict = OrderedDict()
        self.layer_dict["linear1"] = self.linear1
        self.layer_dict["linear2"] = self.linear2
        self.layer_dict["head"] = self.head
        
        # just a sanity check 
        assert id(self.layer_dict["linear1"]) == id(self.linear1)
        
        
        with torch.no_grad():
            if previous_model != None:
                for key in self.layer_dict:
                    self.layer_dict[key].weight_loc.data = previous_model.layer_dict[key].weight_loc.data.clone()
                    self.layer_dict[key].bias_loc.data = previous_model.layer_dict[key].bias_loc.data.clone()
                    self.layer_dict[key].log_weight_scale.data  = previous_model.layer_dict[key].log_weight_scale.data.clone()
                    self.layer_dict[key].log_bias_scale.data  = previous_model.layer_dict[key].log_bias_scale.data.clone()
            if random_initialize: 
                # the first model's initialization
                # 0 mean and very small variances (do not need to break the symmetricity since the training is based on random samples)
                for key in self.layer_dict:
                    self.layer_dict[key].weight_loc.data = torch.randn_like(self.layer_dict[key].weight_loc.data) * 0.1
                    self.layer_dict[key].bias_loc.data = torch.randn_like(self.layer_dict[key].bias_loc.data) * 0.1
                    # initialize to very small value for the first model
                    self.layer_dict[key].log_weight_scale.data  = torch.zeros_like(self.layer_dict[key].log_weight_scale.data) - 10
                    self.layer_dict[key].log_bias_scale.data  = torch.zeros_like(self.layer_dict[key].log_bias_scale.data) - 10
        
        # also save parameters of the previous model, for the calculation of ELBO
        if  previous_model != None:
            previous_locs, previous_logscales = previous_model.get_params()
            previous_locs, previous_logscales = previous_locs.detach().clone(), previous_logscales.detach().clone()
            self.previous_model_locs = previous_locs
            self.previous_model_log_scales = previous_logscales
        else:
            self.previous_model_locs = None
            self.previous_model_log_scales = None
        
    def predict(self, x, n_particles):
        hiddens =  self.linear1.forward(x, x_is_sample=False, activition_fn=nn.ReLU(), n_particles=n_particles)
        hiddens =  self.linear2.forward(hiddens, x_is_sample=True, activition_fn=nn.ReLU(), n_particles=n_particles)
        logits = self.head.forward(hiddens, x_is_sample=True, activition_fn=nn.Identity(), n_particles=n_particles)
        return logits # a list of logits calculated from samples
    
    def predict_MLE(self, x):
        hiddens =  self.linear1.forward_MLE(x, nn.ReLU())
        hiddens =  self.linear2.forward_MLE(hiddens, nn.ReLU())
        logits = self.head.forward_MLE(hiddens, nn.Identity())
        return logits
    
    def get_params(self):  
        locs = []
        logscales = []
        for key in self.layer_dict:
            loc, scale = self.layer_dict[key].get_params()
            locs.append(loc)
            logscales.append(scale)
        locs = torch.cat(locs)
        logscales = torch.cat(logscales)
        return locs, logscales
    
    def calculate_ELBO(self, x, y, n_particles, dataset_size):
        
        locs, logscales = self.get_params()
        # calculate KL between "prior" and posterior
        KL = kl_divergence(Normal(loc=locs, scale=torch.exp(logscales)),
                           Normal(loc=self.previous_model_locs, scale=torch.exp(self.previous_model_log_scales))
                          )
        nELBO = 0 
        for _ in range(n_particles):
            logit = self.predict(x, 1)[0]
            neg_log_p = nn.CrossEntropyLoss(reduction='sum')(logit, y)
            nELBO = neg_log_p + nELBO
        nELBO = nELBO / n_particles / x.shape[0] + KL.sum() / dataset_size 
        # since the ELBO can be viewed as an MC estimator over dataset. Keep the magnitude same is of vital importance!!!
                  
        return nELBO  
    def MLE_loss(self, x, y):
        logit = self.predict_MLE(x)
        loss = nn.CrossEntropyLoss(reduction='mean')(logit, y)
        return loss


In [5]:
ds_test = datasets.MNIST("./", train=False, transform=transforms.ToTensor(), download=True)
ds_train = datasets.MNIST("./", train=True, transform=transforms.ToTensor(), download=True)
def get_pMNIST(task_idx, device):
    np.random.seed(task_idx)
    permute_idx = np.random.choice(28*28, 28*28, False)
    train_x = nn.Flatten()(torch.cat([d[0] for d in ds_train]))[:, permute_idx]
    train_y = torch.tensor([d[1] for d in ds_train])
    
    test_x = nn.Flatten()(torch.cat([d[0] for d in ds_test]))[:, permute_idx]
    test_y = torch.tensor([d[1] for d in ds_test])
    
    return train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [6]:
device = "cuda"

### Compare different coreset size. 

In [7]:
def run_experiments(save_file_name, coreset_config=None):
    previous_model = VCL_permute_MNIST(previous_model=None, random_initialize=False).to(device) # initialize a prior model
    Test_x = []
    Test_y = []

    batch_size = 256

    Accuracies = []

    n_epochs = 200

    coreset = Coreset(coreset_config)

    for task in tqdm(range(10), desc='Running', position=1, leave=False):
        train_x, train_y, test_x, test_y = get_pMNIST(task, device)
        Test_x.append(test_x)
        Test_y.append(test_y)
        
        # define current model
        current_model = VCL_permute_MNIST(previous_model=previous_model, random_initialize=True).to(device)
        current_opt = torch.optim.Adam(current_model.parameters(), lr=0.001)

        ##################################################################################################
        #INSERT YOUR VAE CODES HERE!
        # Trained with train_x, return coreset_idx

        train_x, train_y = coreset.update(train_x, train_y, task)


        ##################################################################################################

        ELBO = []
        for epoch in tqdm(range(n_epochs), desc='Training propagation model', position=2, leave=False):
            ELBO_batch = []
            for batch in range(int(np.ceil(train_x.shape[0] / batch_size))):
                batch_idx0 = batch * batch_size
                batch_idx1 = batch * batch_size + batch_size
                
                current_opt.zero_grad()
                elbo = current_model.calculate_ELBO(x=train_x[batch_idx0: batch_idx1], 
                                                    y=train_y[batch_idx0: batch_idx1], 
                                                    n_particles=1,
                                                    dataset_size=train_x.shape[0])
                elbo.backward()
                nn.utils.clip_grad_value_(current_model.parameters(), 5)
                current_opt.step()
                ELBO_batch.append(elbo.item())
            ELBO.append(np.mean(ELBO_batch))
            
        # calculate prediction model
        pred_model = VCL_permute_MNIST(previous_model=current_model, random_initialize=False).to(device)
        pred_opt = torch.optim.Adam(pred_model.parameters(), lr=0.001)

        ELBO = []
        for epoch in tqdm(range(n_epochs), desc='Training predictive model', position=2, leave=False):
            ELBO_batch = []
            for batch in range(int(np.ceil(coreset.x.shape[0] / batch_size))):
                batch_idx0 = batch * batch_size
                batch_idx1 = batch * batch_size + batch_size
                pred_opt.zero_grad()
                elbo = pred_model.calculate_ELBO(x=coreset.x[batch_idx0: batch_idx1], 
                                                y=coreset.y[batch_idx0: batch_idx1], 
                                                n_particles=1,
                                                dataset_size=coreset.x.shape[0])
                elbo.backward()
                nn.utils.clip_grad_value_(pred_model.parameters(), 5)
                pred_opt.step()
                ELBO_batch.append(elbo.item())
            ELBO.append(np.mean(ELBO_batch))

        test_x_tensor = torch.cat(Test_x)
        test_y_tensor = torch.cat(Test_y)
        pred_y = []
        with torch.no_grad():
            for batch in range(int(np.ceil(test_x_tensor.shape[0] / batch_size))):
                batch_idx0 = batch * batch_size
                batch_idx1 = batch * batch_size + batch_size
                pred_logit_samples = nn.Softmax(-1)(torch.stack(pred_model.predict(test_x_tensor[batch_idx0: batch_idx1], 100), 0)).mean(0)
                pred_y.append(pred_logit_samples.argmax(-1))
            pred_y = torch.cat(pred_y)
            acc = (pred_y == test_y_tensor).cpu().numpy().mean()
        Accuracies.append(acc)
        # print("Task {:d}, Accuracy: {:.4f}".format(task, acc))
        previous_model = current_model


    # print(Accuracies)

    with open(save_file_name, "wb") as f:
        pickle.dump(Accuracies, f)
    return Accuracies

In [8]:
for i in tqdm(range(0,3), desc="Running experiment", position=0):
    # print("====================== Run %d ====================="%i)
    config = CoresetConfig('vae-k-center', None, 200)
    Accuracies = run_experiments(save_file_name="./PermuteMNIST_vae_coreset_k" + "_exps_%d.pkl"%i, coreset_config=config)

Running experiment:   0%|          | 0/3 [00:00<?, ?it/s]

Running:   0%|          | 0/10 [00:00<?, ?it/s]

Running normal VAE


Training propagation model:   0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [29]:
all_accs = np.zeros((10, 10))
for i in range(10):
    file_name = "./PermuteMNIST_vae_coreset_k" + "_exps_%d.pkl"%i
    with open(file_name, 'rb') as f:
        accs = pickle.load( f)
        all_accs[i] = np.array(accs)
        


In [31]:
all_accs.mean(axis=0)

array([0.97643   , 0.973575  , 0.97293667, 0.971915  , 0.9705    ,
       0.968635  , 0.96676857, 0.96444625, 0.96121889, 0.957998  ])

In [32]:
all_accs.std(axis=0)

array([0.0009122 , 0.00134597, 0.00100061, 0.00072588, 0.00114553,
       0.00112048, 0.00108458, 0.00134941, 0.00132648, 0.00136708])

In [ ]:
from google.colab import runtime
runtime.unassign()